# 🇲🇬 Voambolana Malagasy — NLP Project

Ce notebook centralise la collecte de données et la préparation du modèle IA.

### 📑 Étapes d'utilisation :
1.  **Exécutez la cellule de configuration** ci-dessous pour connecter votre Drive et récupérer le code.
2.  **Lancez les scrapers** dans les sections suivantes.
3.  **Les données seront sauvegardées directement sur votre Drive** (dossier `output/`).

## 🛠️ 1. Configuration (Drive + Git)

In [ ]:
from google.colab import drive
import os

# 1. Monter le Drive
print("🔗 Connexion à Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Drive monté avec succès !")
except Exception as e:
    print("❌ Erreur lors du montage du Drive.")

# 2. Se placer dans MyDrive
%cd /content/drive/MyDrive

# 3. Gérer le dépôt Git
REPO_NAME = "nlp_malagasy"
REPO_URL = "https://github.com/franck504/nlp_malagasy.git"

if not os.path.exists(REPO_NAME):
    !git clone $REPO_URL
else:
    %cd $REPO_NAME
    !git pull

if os.path.basename(os.getcwd()) != REPO_NAME:
    %cd $REPO_NAME

print(f"\n📍 Position actuelle : {os.getcwd()}")
!ls

## 🎵 Phase 1 : Découverte des Artistes
Cette étape scanne le site pour trouver tous les artistes disponibles. À faire une seule fois au début.

In [ ]:
!pip install requests beautifulsoup4
%cd /content/drive/MyDrive/nlp_malagasy/tononkira_rehetra

!python 01_discover_artists.py --delay 1.0

## 🎤 Phase 2 : Scraping des Paroles (🚀 TURBO MODE)
Cette étape télécharge les paroles. 


In [ ]:
%cd /content/drive/MyDrive/nlp_malagasy/tononkira_rehetra

# --artist-workers 4 : 4 artistes à la fois
# --song-workers 10  : 10 chansons par artiste en parallèle
!python 02_scrape_lyrics.py --delay 0.5 --artist-workers 4 --song-workers 10

## 📊 Phase 3 : Statistiques
Affiche le nombre de chansons téléchargées et la taille du dataset.

In [ ]:
%cd /content/drive/MyDrive/nlp_malagasy/tononkira_rehetra
!python 03_stats.py

## 📂 Phase 4 : Fusion du Corpus (Finalisation)
Cette étape fusionne les 14,000+ fichiers individuels en **un seul gros fichier texte** propre.
C'est ce fichier qui servira d'entrée pour l'entraînement de l'IA.

In [ ]:
%cd /content/drive/MyDrive/nlp_malagasy/tononkira_rehetra

# Fusionne tout dans 'malagasy_lyrics_corpus.txt'
# On utilise --metadata si on veut garder Titre/Artiste (optionnel)
!python 04_merge_corpus.py --final malagasy_lyrics_corpus.txt

## 🧹 Phase 5 : Purification du Corpus
Cette étape filtre les phrases qui ne sont pas en Malgache (Français, Anglais) et nettoie le texte pour l'IA.
Elle génère le fichier `malagasy_lyrics_cleaned.txt`.

In [ ]:
%cd /content/drive/MyDrive/nlp_malagasy/tononkira_rehetra

!python 05_clean_corpus.py --input malagasy_lyrics_corpus.txt --output malagasy_lyrics_cleaned.txt

## 🧠 3. MVP Modèle IA / NLP

Section pour l'entraînement futur du modèle.